# Lab 6

You are tasked with evaluating card counting strategies for black jack. In order to do so, you will use object oriented programming to create a playable casino style black jack game where a computer dealer plays against $n$ computer players and possibily one human player. If you don't know the rules of blackjack or card counting, please google it. 

A few requirements:
* The game should utilize multiple 52-card decks. Typically the game is played with 6 decks.
* Players should have chips.
* Dealer's actions are predefined by rules of the game (typically hit on 16). 
* The players should be aware of all shown cards so that they can count cards.
* Each player could have a different strategy.
* The system should allow you to play large numbers of games, study the outcomes, and compare average winnings per hand rate for different strategies.

1. Begin by creating a classes to represent cards and decks. The deck should support more than one 52-card set. The deck should allow you to shuffle and draw cards. Include a "plastic" card, placed randomly in the deck. Later, when the plastic card is dealt, shuffle the cards before the next deal.

In [5]:
deck=list()
for suite in ("Clubs","Diamonds","Hearts","Spades"):
    for values in list(range(2,10))+["Ace","King","Queen","Jack"]:
        deck.append((suite,values))

In [6]:
deck

[('Clubs', 2),
 ('Clubs', 3),
 ('Clubs', 4),
 ('Clubs', 5),
 ('Clubs', 6),
 ('Clubs', 7),
 ('Clubs', 8),
 ('Clubs', 9),
 ('Clubs', 'Ace'),
 ('Clubs', 'King'),
 ('Clubs', 'Queen'),
 ('Clubs', 'Jack'),
 ('Diamonds', 2),
 ('Diamonds', 3),
 ('Diamonds', 4),
 ('Diamonds', 5),
 ('Diamonds', 6),
 ('Diamonds', 7),
 ('Diamonds', 8),
 ('Diamonds', 9),
 ('Diamonds', 'Ace'),
 ('Diamonds', 'King'),
 ('Diamonds', 'Queen'),
 ('Diamonds', 'Jack'),
 ('Hearts', 2),
 ('Hearts', 3),
 ('Hearts', 4),
 ('Hearts', 5),
 ('Hearts', 6),
 ('Hearts', 7),
 ('Hearts', 8),
 ('Hearts', 9),
 ('Hearts', 'Ace'),
 ('Hearts', 'King'),
 ('Hearts', 'Queen'),
 ('Hearts', 'Jack'),
 ('Spades', 2),
 ('Spades', 3),
 ('Spades', 4),
 ('Spades', 5),
 ('Spades', 6),
 ('Spades', 7),
 ('Spades', 8),
 ('Spades', 9),
 ('Spades', 'Ace'),
 ('Spades', 'King'),
 ('Spades', 'Queen'),
 ('Spades', 'Jack')]

2. Now design your game on a UML diagram. You may want to create classes to represent, players, a hand, and/or the game. As you work through the lab, update your UML diagram. At the end of the lab, submit your diagram (as pdf file) along with your notebook. 

3. Begin with implementing the skeleton (ie define data members and methods/functions, but do not code the logic) of the classes in your UML diagram.

In [7]:
class base:
    SILENT=6
    DEBUG=1
    INFO=2
    WARNING=3
    ERROR=4
    CRITICAL=5
    
    def __init__(self,level=0):
        self.level=level
        
    def message(self,level,*args):
        if level >= self.level:
            print(*args)

In [8]:
class Card(base):
    __suits = ["Clubs", "Diamonds", "Hearts", "Spades", "ShuffleCard"]
    __values = list(range(2,11)) + [ "Jack", "Queen", "King", "Ace"]

    def __init__(self,suit,value=None):
        base.__init__(self)
        self.__suit = suit if suit in self.__suits else None
        self.__value = value if value in self.__values else None
        
        if self.__suit is None:
            self.message(self.ERROR, "Error, bad suit:",suit)

        if self.__value is None and self.__suit != "ShuffleCard":
            self.message(self.ERROR, "Error, bad value:",value)

    def value(self):
        return self.__value
    
    def suit(self):
        return self.__suit
    
    def numerical_value(self):
        # Special Handling of aces
        if self.__value == "Ace":
            return 1
        elif self.__value in [ "Jack", "Queen", "King"]:
            return 10
        else:
            return self.__value
        
    def shuffle_card(self):
        return self.__suit == "ShuffleCard"

    def __str__(self):
        if self.shuffle_card():
            return "Shuffle Card"
        else:
            return str(self.__value) + " of " + self.__suit

    __repr__ = __str__

In [9]:
import random

class Deck(base):
    __suits = ["Clubs", "Diamonds", "Hearts", "Spades"]
    __values = list(range(2,11)) + [ "Jack", "Queen", "King", "Ace"]

    def __init__(self,n_decks=6):
        base.__init__(self)
        self.__n_decks=n_decks
        
        self.__cards = list()
        
        for _ in range(self.__n_decks):
            self.__cards.extend(self.__make_deck())            
            
        # TODO: Add logic to appropriately place shufflecard
        self.__cards.append(Card("ShuffleCard"))
        
    def __make_deck(self):
        deck=list()
        for suit in self.__suits:
            for value in self.__values:
                deck.append(Card(suit,value))
        return deck
    
    def shuffle(self):
        random.shuffle(self.__cards) 
        
    def deal(self):
        if len(self.__cards)>0:
            return self.__cards.pop()
        else:
            for _ in range(self.__n_decks):
                self.__cards.extend(self.__make_deck()) 
            self.shuffle()
            return self.__cards.pop()       

In [10]:
my_deck = Deck()
my_deck.shuffle()

[my_deck.deal() for _ in range(10)]

[8 of Hearts,
 Jack of Spades,
 5 of Clubs,
 5 of Hearts,
 King of Diamonds,
 King of Diamonds,
 8 of Spades,
 3 of Diamonds,
 4 of Hearts,
 9 of Hearts]

In [11]:
def cacluate_hand_value(hand):
    return sum(map(lambda x: x.numerical_value(),hand))

In [12]:
cacluate_hand_value([my_deck.deal() for _ in range(10)])

78

In [13]:
class PlayerBase(base):
    def __init__(self,name,chips):
        base.__init__(self)
        self.__name=name
        self.__chips=chips
        self.__hand=list()
    
    def name(self):
        return self.__name
    
    def chips(self):
        return self.__chips

    def pay(self,amount):
        self.__chips+=amount

    def deduct(self,chips):
        self.__chips-=chips

    def play_hand(self, down_card, up_card, seen_card):
          raise NotImplementedError

    def __str__(self):
        return self.__name
    
    __repr__ = __str__

class Dealer(PlayerBase):
    def __init__(self,threshold):
        self._threshold(self,threshold=16)
        PlayerBase.__init__(self,"Dealer",100)

    def play_hand(self, down_card, up_card, seen_card):
        hand_value=cacluate_hand_value([down_card]+ up_card)
        return hand_value < self.__threshold

class ConsolePlayer(PlayerBase):
      def play_hand(self, down_card, up_card, seen_card):
        print("Down Card:", down_card)
        print("Up Card:", up_card)
        print("Seen Card:", seen_card)
        hit = input("Hit? (y/n)")
        return hit == "y"

class StrategyPlayer(PlayerBase):
      def play_hand(self, down_card, up_card, seen_card):
              return True

In [31]:
class Game(base):
      def _init_(self,n_decks=6):
        base._init_(self,self.INFO)
        self.__n_decks=n_decks
        self.__deck=Deck(n_decks)
        self.__all_players=list()

        self.__shuffle=False

        def players(self):
            return self.__players

        def add_player(self,player):
            self.__all_players.append(player)

        def deal_and_check_shuffle(self,deck):
            card=deck.deal()
            if card.shuffle_card():
                shuffle=True
                card=deck.deal()
            return card

        def show_status(self,hands,seen_cards):
            self.message(self.INFO)
            self.message(self.INFO,"Hands:",hands)
            self.message(self.INFO,"Seen Cards:",seen_cards)
            self.message(self.INFO,"Players:")
            for i,player in enumerate(self.__all_players):
                self.message(self.INFO,i,":",player)
                self.message(self.INFO)

        def play_game(self,n_hands):
              self.add_player(Dealer())
               
                deck = None 
                self.__shuffle=False

                for i_hand in range(n_hands):
                    self.__players=list(filter(lambda player: player.chips()>=2,self._players))
                    self.message(self.DEBUG,"players, all players",len(self.__players),len(self.__all_players))
                    self.message(self.DEBUG,"Starting Hand",i_hand,n_hands)

                if deck is None or self.__shuffle:
                    self.message(self.DEBUG,"Creating new deck")
                    deck=Deck()
                    deck.shuffle()
                    seen_cards=list()
                    self.__shuffle=False

                hands=list()

                self.message(self.DEBUG,"Dealing Hand")
                for player_i,player in enumerate(self.__players):
                      down_card=self.deal_and_check_shuffle(deck)
                          up_card=list()
                              hands.append((down_card,up_card))
                                  if player_i < len(self.__players)-1:
                seen_cards.append(down_card)


IndentationError: unexpected indent (785770373.py, line 35)

4. Complete the implementation by coding the logic of all functions. For now, just implement the dealer player and human player.

5.  Test. Demonstrate game play. For example, create a game of several dealer players and show that the game is functional through several rounds.

6. Implement a new player with the following strategy:

    * Assign each card a value: 
        * Cards 2 to 6 are +1 
        * Cards 7 to 9 are 0 
        * Cards 10 through Ace are -1
    * Compute the sum of the values for all cards seen so far.
    * Hit if sum is very negative, stay if sum is very positive. Select a threshold for hit/stay, e.g. 0 or -2.  

7. Create a test scenario where one player, using the above strategy, is playing with a dealer and 3 other players that follow the dealer's strategy. Each player starts with same number of chips. Play 50 rounds (or until the strategy player is out of money). Compute the strategy player's winnings. You may remove unnecessary printouts from your code (perhaps implement a verbose/quiet mode) to reduce the output.

8. Create a loop that runs 100 games of 50 rounds, as setup in previous question, and store the strategy player's chips at the end of the game (aka "winnings") in a list. Histogram the winnings. What is the average winnings per round? What is the standard deviation. What is the probabilty of net winning or lossing after 50 rounds?


9. Repeat previous questions scanning the value of the threshold. Try at least 5 different threshold values. Can you find an optimal value?

10. Create a new strategy based on web searches or your own ideas. Demonstrate that the new strategy will result in increased or decreased winnings. 